In [9]:
with open("../config/app_id.txt") as f:
    app_id = f.read()

In [10]:
import requests as req
import pandas as pd
import numpy as np
import json

In [11]:
link = f"https://api.worldoftanks.ru/wot/encyclopedia/vehicles/?application_id={app_id}&limit=1"


In [12]:
data = json.loads(req.get(link).text)


In [ ]:
data

In [15]:
len(data["data"])

713

In [16]:
df = pd.DataFrame()
try:
    for tank_id in data["data"]:
        #tank_id = "13889" 

        tank = {}
        tank["tag"] = data["data"][tank_id]["tag"]
        tank["lvl"] = data["data"][tank_id]["tier"]
        tank["type"] = data["data"][tank_id]["type"]

        modules = {}
        modules["vehicleTurret"] = []
        modules["vehicleGun"] = []
        for i in data["data"][tank_id]["modules_tree"]:
            if data["data"][tank_id]["modules_tree"][i]["type"] == "vehicleTurret":
                modules["vehicleTurret"].append(i)
            elif data["data"][tank_id]["modules_tree"][i]["type"] == "vehicleGun":
                    modules["vehicleGun"].append(i)
            elif data["data"][tank_id]["modules_tree"][i]["next_modules"] is None:
                    modules[data["data"][tank_id]["modules_tree"][i]["type"]] = i

        if len(modules["vehicleTurret"]) > 1:
            if data["data"][tank_id]["modules_tree"][modules["vehicleTurret"][0]]['is_default']:
                modules["vehicleTurret"] = modules["vehicleTurret"][1]
            else:
                modules["vehicleTurret"] = modules["vehicleTurret"][0]
        elif len(modules["vehicleTurret"]) == 0:
            modules["vehicleTurret"] = None
        else:
            modules["vehicleTurret"] = modules["vehicleTurret"][0]

        config = {}
        dpm = 0

        for i in modules["vehicleGun"]:
            link = f"https://api.worldoftanks.ru/wot/encyclopedia/vehicleprofile/?application_id={app_id}"
            link += "&tank_id=" + tank_id
            link += "&gun_id=" + i
            link += "&suspension_id=" + modules["vehicleChassis"]
            link += "&engine_id=" + modules["vehicleEngine"]
            link += "&radio_id=" + modules["vehicleRadio"]
            if modules["vehicleTurret"] is not None:
                link += "&turret_id=" + modules["vehicleTurret"]
            tmp_config = json.loads(req.get(link).text)
            
            if tmp_config["status"] == "error":
                continue
            tmp_dpm = tmp_config["data"][tank_id]["ammo"][0]["damage"][1] * tmp_config["data"][tank_id]["gun"]["fire_rate"]
            
            if dpm < tmp_dpm:
                dpm = tmp_dpm
                config = tmp_config

        tank["damage"] = config["data"][tank_id]["ammo"][0]["damage"][1]
        tank["dpm"] = round(dpm, 2)
        tank["penetration"] = config["data"][tank_id]["ammo"][0]["penetration"][1]
        tank["accuracy"] = config["data"][tank_id]["gun"]["dispersion"]
        tank["aim_time"] = config["data"][tank_id]["gun"]["aim_time"]
        tank["speed"] = config["data"][tank_id]["speed_forward"]
        tank["cp_power"] = round(config["data"][tank_id]["engine"]["power"] / (config["data"][tank_id]["weight"] / 1000), 2)
        tank["hp"] = config["data"][tank_id]["hp"]
        tank["armor_hull"] = config["data"][tank_id]["armor"]["hull"]["front"]
        if config["data"][tank_id]["armor"]["turret"] is not None:
            tank["armor_turret"] = config["data"][tank_id]["armor"]["turret"]["front"]
            tank["view_range"] = config["data"][tank_id]["turret"]["view_range"]
        else:
            tank["armor_turret"] = np.nan
        tank["view_range"] = config["data"][tank_id]["turret"]["view_range"]

        df = pd.concat([df, pd.DataFrame(tank, index=[tank_id])])
except Exception as e:
    print(tank_id)
    print(str(e))
    print(modules)
    print(config)
    
#df

In [ ]:
df

,tag,lvl,type,damage,dpm,penetration,accuracy,aim_time,speed,cp_power,hp,armor_hull,armor_turret,view_range
1,R04_T-34,5,mediumTank,85,2217.65,112,0.34,2.3,56,17.40,590,45,52.0,350
3137,F08_AMX_50_100,8,heavyTank,240,1776.00,212,0.36,3.0,51,17.11,1400,90,90.0,380
2049,R12_A-20,5,lightTank,40,2168.80,46,0.45,2.3,72,28.39,560,20,37.0,360
15617,R95_Object_907,10,mediumTank,320,2953.60,264,0.36,2.0,55,16.94,2000,110,228.0,400
49169,G136_Tiger_131,6,heavyTank,220,1885.40,145,0.38,2.3,40,11.40,1060,100,100.0,370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,A01_T1_Cunningham,1,lightTank,30,900.00,33,0.51,2.5,41,17.47,245,10,10.0,280
4705,J21_Type_91,3,heavyTank,70,1999.90,81,0.40,2.1,25,12.44,445,20,20.0,330
47873,R143_T_29,3,mediumTank,110,1410.20,66,0.55,2.3,45,17.54,445,30,20.0,300
1841,Ch18_WZ-120,9,mediumTank,440,2398.00,249,0.38,3.4,60,16.13,1750,100,220.0,390


In [ ]:
df.to_excel("../data/asserts/tanks.xlsx")